In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from scipy import sparse
from collections import Counter
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import nltk
from nltk.corpus import stopwords
import string

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
import re
def get_good_tokens(sentence):
    replaced_punctation = list(map(lambda token: re.sub('[^0-9A-Za-z!?]+', '', token), sentence))
    removed_punctation = list(filter(lambda token: token, replaced_punctation))
    return removed_punctation

In [4]:
import copy
def w2v_preprocessing(df_in):
    """ All the preprocessing steps for word2vec are done in this function.
    All mutations are done on the dataframe itself. So this function returns
    nothing.
    """
    df = copy.deepcopy(df_in)
    df['text'] = df.text.str.lower()
    df['document_sentences'] = df.text.str.split('.')  # split texts into individual sentences
    df['tokenized_sentences'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         df.document_sentences))  # tokenize sentences
    df['tokenized_sentences'] = list(map(lambda sentences:
                                         list(map(get_good_tokens, sentences)),
                                         df.tokenized_sentences))  # remove unwanted characters
    df['tokenized_sentences'] = list(map(lambda sentences:
                                         list(filter(lambda lst: lst, sentences)),
                                         df.tokenized_sentences))  # remove empty lists
    return df

train = w2v_preprocessing(train_data)
test = w2v_preprocessing(test_data)

In [6]:
train[['id','label','tokenized_sentences']].to_csv("clean.csv")

In [8]:
sentences = []
from gensim.models import Word2Vec
for sentence_group in train.tokenized_sentences:
    sentences.extend(sentence_group)
print(sentences[0:10])

[['nice', 'hotel', 'expensive', 'parking', 'got', 'good', 'deal', 'stay', 'hotel', 'anniversary'], ['arrived', 'late', 'evening', 'took', 'advice', 'previous', 'reviews', 'did', 'valet', 'parking'], ['check', 'quick', 'easy'], ['little', 'disappointed', 'nonexistent', 'view', 'room', 'room', 'clean', 'nice', 'size'], ['bed', 'comfortable', 'woke', 'stiff', 'neck', 'high', 'pillows'], ['not', 'soundproof', 'like', 'heard', 'music', 'room', 'night', 'morning', 'loud', 'bangs', 'doors', 'opening', 'closing', 'hear', 'people', 'talking', 'hallway'], ['maybe', 'just', 'noisy', 'neighbors'], ['aveda', 'bath', 'products', 'nice'], ['did', 'not', 'goldfish', 'stay', 'nice', 'touch', 'taken', 'advantage', 'staying', 'longer'], ['location', 'great', 'walking', 'distance', 'shopping']]


In [9]:
sentences = []
from gensim.models import Word2Vec
for sentence_group in train.tokenized_sentences:
    sentences.extend(sentence_group)

# Set values for various parameters
num_features = 200    # Word vector dimensionality
min_word_count = 3    # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 6           # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model
W2Vmodel = Word2Vec(sentences=sentences,
                    sg=1,
                    hs=0,
                    workers=num_workers,
                    size=num_features,
                    min_count=min_word_count,
                    window=context,
                    sample=downsampling,
                    negative=5,
                    iter=6)

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


X_val=train_data.sample(frac=0.2,random_state=200)
X_train=train_data.drop(X_val.index)
texts=X_train.text

NUM_WORDS=80000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(texts)
sequences_valid=tokenizer.texts_to_sequences(X_val.text)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 39479 unique tokens.


In [8]:

y_train = to_categorical(np.asarray(train_data.label[X_train.index]))
y_val = to_categorical(np.asarray(train_data.label[X_val.index]))
X_train = pad_sequences(sequences_train)
X_val = pad_sequences(sequences_valid,maxlen=X_train.shape[1])
print('Shape of X train and X validation tensor:', X_train.shape,X_val.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)

Shape of X train and X validation tensor: (12800, 2039) (3200, 2039)
Shape of label train and validation tensor: (12800, 6) (3200, 6)


In [9]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = W2Vmodel.wv

EMBEDDING_DIM=200
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [10]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = X_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5



inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=6, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)

In [13]:
adam = Adam(lr=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss')]
model.fit(X_train, y_train, epochs=5, verbose=1, validation_data=(X_val, y_val),
         callbacks=callbacks)  # starts training

Train on 12800 samples, validate on 3200 samples
Epoch 1/5
12800/12800 [==============================] - 1102s 86ms/step - loss: 1.2799 - acc: 0.5483 - val_loss: 1.1331 - val_acc: 0.6059
Epoch 2/5
12800/12800 [==============================] - 1087s 85ms/step - loss: 1.1003 - acc: 0.6255 - val_loss: 1.0990 - val_acc: 0.6006
Epoch 3/5
12800/12800 [==============================] - 1091s 85ms/step - loss: 1.0396 - acc: 0.6528 - val_loss: 1.0798 - val_acc: 0.6219
Epoch 4/5
12800/12800 [==============================] - 1085s 85ms/step - loss: 0.9927 - acc: 0.6783 - val_loss: 1.0752 - val_acc: 0.6300
Epoch 5/5
12800/12800 [==============================] - 1110s 87ms/step - loss: 0.9506 - acc: 0.7119 - val_loss: 1.0925 - val_acc: 0.6300


In [14]:
sequences_test=tokenizer.texts_to_sequences(test_data.text)
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
y_pred=model.predict(X_test)

In [21]:
X_test

array([[    0,     0,     0, ...,     7,   141,   120],
       [    0,     0,     0, ...,     1,  2942,    86],
       [    0,     0,     0, ...,  8276, 20149,    45],
       ...,
       [    0,     0,     0, ...,   421,  1523,  1047],
       [    0,     0,     0, ...,   326,  6470,  3940],
       [    0,     0,     0, ...,   239,   151,    26]], dtype=int32)

In [46]:
y_pred_label = np.argmax(y_pred, axis=1)

In [47]:
df_sub = pd.DataFrame()
df_sub['id']=test_data.id

In [52]:
df_sub['pred']=y_pred_label

In [55]:
df_sub=df_sub[['id','pred']]

In [56]:
df_sub.to_csv("deep.csv", index=False)